In [1]:
%cd ../..

/home/work/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
target_actor = fep_control.MountainCarActor(state_dim=2, batch_shape=(1,), trainable=False)
target_observer = fep_control.GenerativeObserver(observation_dim=2, batch_shape=(1,), trainable=False)

proposal_actor = fep_control.RecognitionActor(state_dim=2, observation_dim=2, batch_shape=(1,), discrete_actions=False,
                                              name='MountainCarActor', trainable=True)
proposal_encoder = fep_control.RecognitionEncoder(state_dim=2, observation_dim=2, batch_shape=(1,), trainable=True)

actor = importance.propose(target_actor, proposal_actor)
observer = importance.propose(target_observer, proposal_encoder)

agent = compose(observer, actor)

In [5]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [6]:
theta, graph, log_weight = active.active_inference(agent, 'MountainCarContinuous-v0', lr=1e-2, episodes=10, episode_length=1000, dream=False)

05/31/2019 12:00:51 ELBO=-2.64198496e+04 at episode 0 of length 1000
05/31/2019 12:00:57 ELBO=-1.03526279e+04 at episode 1 of length 317
05/31/2019 12:01:05 ELBO=-2.73290594e+05 at episode 2 of length 1000
05/31/2019 12:01:13 ELBO=-1.97716191e+04 at episode 3 of length 1000
05/31/2019 12:01:21 ELBO=-2.31757891e+04 at episode 4 of length 1000
05/31/2019 12:01:28 ELBO=-1.62074121e+04 at episode 5 of length 180
05/31/2019 12:01:36 ELBO=-8.67365430e+03 at episode 6 of length 186
05/31/2019 12:01:43 ELBO=-7.21401562e+03 at episode 7 of length 240
05/31/2019 12:01:51 ELBO=-6.83507812e+03 at episode 8 of length 247
05/31/2019 12:01:58 ELBO=-5.72624561e+03 at episode 9 of length 135


In [7]:
log_weight

tensor([-5726.2456], grad_fn=<AddBackward0>)

In [8]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [9]:
theta

(tensor([[-2.1389, -0.5148]], grad_fn=<AddBackward0>),
 tensor([[0.3280]], grad_fn=<AddBackward0>))

In [10]:
log_weight

tensor([-7796.7012], grad_fn=<AddBackward0>)

In [11]:
torch.save(agent, 'examples/fep_control/fep_mountain_car_agent.dat')